In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tqdm, os
import tifffile
import glob

In [2]:
f = glob.glob('D:/allen/data/mydata/ac_time_seperated/20220412/merged/*/*.csv')

In [3]:
f

['D:/allen/data/mydata/ac_time_seperated/20220412/merged\\1\\track.csv',
 'D:/allen/data/mydata/ac_time_seperated/20220412/merged\\2\\track.csv',
 'D:/allen/data/mydata/ac_time_seperated/20220412/merged\\3\\track.csv',
 'D:/allen/data/mydata/ac_time_seperated/20220412/merged\\4\\track.csv']

In [4]:
# Path to the folder containing the single channel Hoechst microscopy image used for TrackMate. It is important that
# nothing else is present in this folder.
date = '20220412'
scence = '4'

image_folder = r'D:/allen/data/mydata/ac_time_seperated/'+ date +'/merged/' + scence + '/'

# Path to the tracking csv file (rfptrack.cav or gfptrack.csv) generated by TrackMate.
csv_p = image_folder + r"track.csv"

# Path of the track curation folder which will contain the visualization of the tracks with a potential cell division.
track_curation_folder = image_folder + r'crop/'

# Track curation: for each track creates images of the detected cells in a folder, to be manually curated for cell division

filenames = os.listdir(image_folder)

saved_folder = r'D:/YR_classification/data/notlabeled/' + date + '/' + scence + '/'

if not os.path.exists(saved_folder):
    os.makedirs(saved_folder)

In [5]:
filenames

['4.tif', 'track.csv']

In [7]:
img = tifffile.imread(image_folder + filenames[0])[:,1:,:,:]

In [8]:

# Define the number of frames to be used
n_frames = img.shape[0]

df = pd.read_csv(csv_p)

# Loop over all tracks and store the track id with potential division in 'split_track.npy'.
if not os.path.exists(track_curation_folder):
    os.mkdir(track_curation_folder)

In [9]:
df

,Label,ID,TRACK_ID,QUALITY,POSITION_X,POSITION_Y,POSITION_Z,POSITION_T,FRAME,RADIUS,...,MANUAL_COLOR,MEAN_INTENSITY,MEDIAN_INTENSITY,MIN_INTENSITY,MAX_INTENSITY,TOTAL_INTENSITY,STANDARD_DEVIATION,ESTIMATED_DIAMETER,CONTRAST,SNR
0,ID320,320,None,2.170,0.00,89.44,0,0,0,15,...,-10921639,50.119,0.0,0,255,537922.336,99.733,14.677,0.870,0.468
1,ID321,321,4,2.062,118.56,92.56,0,0,0,15,...,-10921639,118.389,0.0,0,255,1270665.000,127.180,17.896,0.581,0.684
2,ID322,322,40,1.660,172.64,96.72,0,0,0,15,...,-10921639,119.434,0.0,0,255,1281885.000,127.251,57.000,0.225,0.345
3,ID323,323,89,1.873,245.44,114.40,0,0,0,15,...,-10921639,98.503,0.0,0,255,1057230.000,124.165,11.611,0.847,0.727
4,ID324,324,None,1.987,138.32,115.44,0,0,0,15,...,-10921639,100.879,0.0,0,255,1082730.000,124.696,15.109,0.558,0.579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1701,ID1723,1723,None,1.059,265.20,61.36,0,67,67,15,...,-10921639,41.363,0.0,0,255,443946.377,93.259,18.017,0.831,0.403
1702,ID1724,1724,257,1.441,202.80,75.92,0,67,67,15,...,-10921639,71.632,0.0,0,255,768825.000,114.613,11.628,0.420,0.369
1703,ID1725,1725,None,1.922,170.56,76.96,0,67,67,15,...,-10921639,98.313,0.0,0,255,1055190.000,124.120,57.000,0.473,0.509
1704,ID1726,1726,None,1.472,153.92,81.12,0,67,67,15,...,-10921639,83.179,0.0,0,255,892755.000,119.554,57.000,0.545,0.491


In [10]:
# If the 'TRACK_ID' is not string-type, the ID wilee be ignored in the next part.
for i in range(df.shape[0]):
    df.loc[i,'TRACK_ID'] = str(df.loc[i,'TRACK_ID'])

In [11]:
if not os.path.exists(track_curation_folder + 'split_track.npy'):
    split_track = []
    for track in tqdm.tqdm(np.unique(df['TRACK_ID'])):
        if track != 'None':
            print('track:',track)
            arr = df[df['TRACK_ID'] == str(track)]['FRAME']
            print('arr:',arr)
            frames, nID = np.unique(arr, return_counts=True)
            print(nID)
            division_frame = np.where(nID >= 1)
            #print('division_frame:',division_frame)
            if len(division_frame[0]) > 0:
                if nID[division_frame[0][0]] == nID[-1]:
                    split_track = np.append(split_track, track)
    np.save(track_curation_folder + 'split_track.npy', split_track)
else:
    split_track = np.load(track_curation_folder + 'split_track.npy')

 36%|████████████████████████████                                                  | 103/286 [00:00<00:00, 1022.54it/s]

track: 0
arr: 20      0
58      1
61      2
103     3
144     4
167     5
204     6
210     7
258     8
284     9
300    10
Name: FRAME, dtype: int64
[1 1 1 1 1 1 1 1 1 1 1]
track: 1
arr: 169    5
177    6
231    8
262    9
Name: FRAME, dtype: int64
[1 1 1 1]
track: 10
arr: 9       0
49      1
78      2
90      3
135     4
159     5
194     6
228     7
247     8
273     9
287    10
334    11
357    12
380    13
407    14
433    15
469    16
494    17
518    18
551    19
Name: FRAME, dtype: int64
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
track: 100
arr: 539    19
570    20
623    21
634    22
680    23
699    24
739    26
792    27
Name: FRAME, dtype: int64
[1 1 1 1 1 1 1 1]
track: 101
arr: 541    19
573    20
626    21
635    22
683    23
700    24
714    25
743    26
766    27
Name: FRAME, dtype: int64
[1 1 1 1 1 1 1 1 1]
track: 102
arr: 546    19
578    20
Name: FRAME, dtype: int64
[1 1]
track: 103
arr: 556    19
587    20
610    21
Name: FRAME, dtype: int64
[1 1 1]
track: 104
arr: 5

100%|███████████████████████████████████████████████████████████████████████████████| 286/286 [00:00<00:00, 931.06it/s]

[1 1 1 1]
track: 264
arr: 1333    50
1372    51
1392    52
1408    53
Name: FRAME, dtype: int64
[1 1 1 1]
track: 265
arr: 1393    52
1412    53
Name: FRAME, dtype: int64
[1 1]
track: 266
arr: 1383    52
1401    53
1444    54
Name: FRAME, dtype: int64
[1 1 1]
track: 267
arr: 1425    54
1451    55
Name: FRAME, dtype: int64
[1 1]
track: 268
arr: 1428    54
1455    55
Name: FRAME, dtype: int64
[1 1]
track: 269
arr: 1429    54
1457    55
Name: FRAME, dtype: int64
[1 1]
track: 27
arr: 446    15
485    16
507    17
Name: FRAME, dtype: int64
[1 1 1]
track: 270
arr: 1435    54
1462    55
Name: FRAME, dtype: int64
[1 1]
track: 271
arr: 1404    53
1427    54
Name: FRAME, dtype: int64
[1 1]
track: 272
arr: 1463    55
1483    56
Name: FRAME, dtype: int64
[1 1]
track: 273
arr: 1540    59
1562    60
Name: FRAME, dtype: int64
[1 1]
track: 274
arr: 1541    59
1563    60
Name: FRAME, dtype: int64
[1 1]
track: 275
arr: 1494    57
1524    58
Name: FRAME, dtype: int64
[1 1]
track: 276
arr: 1498    57
1528 

In [12]:
split_track

array(['0', '1', '10', '100', '101', '102', '103', '104', '105', '106',
       '107', '108', '109', '11', '110', '111', '112', '113', '114',
       '115', '116', '117', '118', '119', '12', '120', '121', '122',
       '123', '124', '125', '126', '127', '128', '129', '13', '130',
       '131', '132', '133', '134', '135', '136', '137', '138', '139',
       '14', '140', '141', '142', '143', '144', '145', '146', '147',
       '148', '149', '15', '150', '151', '152', '153', '154', '155',
       '156', '157', '158', '159', '16', '160', '161', '162', '163',
       '164', '165', '166', '167', '168', '169', '17', '170', '171',
       '172', '173', '174', '175', '176', '177', '178', '179', '18',
       '180', '181', '182', '183', '184', '185', '186', '187', '188',
       '189', '19', '190', '191', '192', '193', '194', '195', '196',
       '197', '198', '199', '2', '20', '200', '201', '202', '203', '204',
       '205', '206', '207', '208', '209', '21', '210', '211', '212',
       '213', '214', '21

In [13]:
factor = 3.8496 # 1024/266
crop_size = 70 # half
image_width = 1024 # resolution
for track_id in split_track:
    if not os.path.exists(saved_folder + '{}'.format(track_id)):
        os.makedirs(saved_folder + '{}'.format(track_id))

        print(track_id)

        # Get extreme coordinates of the track for image dimensions
        x_min, x_max = np.int(np.min(df[df['TRACK_ID'] == track_id]['POSITION_X'].values*factor)), \
                       np.int(np.max(df[df['TRACK_ID'] == track_id]['POSITION_X'].values*factor))
        y_min, y_max = np.int(np.min(df[df['TRACK_ID'] == track_id]['POSITION_Y'].values*factor)), \
                       np.int(np.max(df[df['TRACK_ID'] == track_id]['POSITION_Y'].values*factor))

        if np.min([x_min, x_max, y_min, y_max]) > crop_size and np.max([x_min, x_max, y_min, y_max]) < (image_width-crop_size):
            for frame in tqdm.tqdm(np.unique(df[df['TRACK_ID'] == track_id]['FRAME'].values)):
                x = np.int(df[df['TRACK_ID'] == track_id][df['FRAME'] == frame]['POSITION_X'].values*factor)
                y = np.int(df[df['TRACK_ID'] == track_id][df['FRAME'] == frame]['POSITION_Y'].values*factor)
                im_bf = img[frame-1,:,y - crop_size:y + crop_size,x - crop_size:x + crop_size]
                print('im_bf:',im_bf.shape)
                tifffile.imwrite(saved_folder + '{}/{}.tiff'.format(track_id, frame),im_bf,photometric='minisblack')

        else:
            for frame in tqdm.tqdm(np.unique(df[df['TRACK_ID'] == track_id]['FRAME'].values)):
                x = np.int(df[df['TRACK_ID'] == track_id][df['FRAME'] == frame]['POSITION_X'].values*factor)
                y = np.int(df[df['TRACK_ID'] == track_id][df['FRAME'] == frame]['POSITION_Y'].values*factor)
                if x > crop_size and y < (image_width-crop_size) and x < (image_width-crop_size) and y > crop_size:
                    im_bf = img[frame-1,:,y - crop_size:y + crop_size,x - crop_size:x + crop_size]
                    print('im_bf:',im_bf.shape)
                    tifffile.imwrite(saved_folder + '{}/{}.tiff'.format(track_id, frame),im_bf,photometric='minisblack')
                elif x < 0.5*crop_size or x > (image_width-0.5*crop_size) or y < 0.5*crop_size or y > (image_width-0.5*crop_size):
                    pass  # get rid of cells too close to edge
                elif x > crop_size and x < (image_width-crop_size) and y > (image_width-crop_size):
                    im_bf = img[frame-1,:,image_width-crop_size*2:image_width,x - crop_size:x + crop_size]
                    print('im_bf:',im_bf.shape)
                    tifffile.imwrite(saved_folder + '{}/{}.tiff'.format(track_id, frame),im_bf,photometric='minisblack')
                elif x > crop_size and x < (image_width-crop_size) and y < crop_size:
                    im_bf = img[frame-1,:,0:crop_size*2,x - crop_size:x + crop_size]
                    print('im_bf:',im_bf.shape)
                    tifffile.imwrite(saved_folder + '{}/{}.tiff'.format(track_id, frame),im_bf,photometric='minisblack')
                elif x < crop_size and y < (image_width-crop_size) and y > crop_size:
                    im_bf = img[frame-1,:,y - crop_size:y + crop_size, 0:crop_size*2]
                    print('im_bf:',im_bf.shape)
                    tifffile.imwrite(saved_folder + '{}/{}.tiff'.format(track_id, frame),im_bf,photometric='minisblack')
                elif x > (image_width-crop_size) and y < (image_width-crop_size) and y > crop_size:
                    im_bf = img[frame-1,:,y - crop_size:y + crop_size, image_width -crop_size*2:image_width]
                    print('im_bf:',im_bf.shape)
                    tifffile.imwrite(saved_folder + '{}/{}.tiff'.format(track_id, frame),im_bf,photometric='minisblack')
                elif x < crop_size and y < crop_size:
                    im_bf = img[frame-1,:,0:crop_size*2, 0:crop_size*2]
                    print('im_bf:',im_bf.shape)
                    tifffile.imwrite(saved_folder + '{}/{}.tiff'.format(track_id, frame),im_bf,photometric='minisblack')
                elif x < crop_size and y > (image_width-crop_size):
                    im_bf = img[frame-1,:,image_width-crop_size*2:image_width, 0:crop_size*2]
                    print('im_bf:',im_bf.shape)
                    tifffile.imwrite(saved_folder + '{}/{}.tiff'.format(track_id, frame),im_bf,photometric='minisblack')
                elif x > (image_width-crop_size) and y < crop_size:
                    im_bf = img[frame-1,:,0:crop_size*2, image_width-crop_size*2:image_width]
                    print('im_bf:',im_bf.shape)
                    tifffile.imwrite(saved_folder + '{}/{}.tiff'.format(track_id, frame),im_bf,photometric='minisblack')
                else:
                    im_bf = img[frame-1,:,image_width-crop_size*2:image_width, image_width-crop_size*2:image_width]
                    print('im_bf:',im_bf.shape)
                    tifffile.imwrite(saved_folder + '{}/{}.tiff'.format(track_id, frame),im_bf,photometric='minisblack')

  0%|                                                                                           | 0/11 [00:00<?, ?it/s]<ipython-input-13-d6b00e662556>:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  x = np.int(df[df['TRACK_ID'] == track_id][df['FRAME'] == frame]['POSITION_X'].values*factor)
<ipython-input-13-d6b00e662556>:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  y = np.int(df[df['TRACK_ID'] == track_id][df['FRAME'] == frame]['POSITION_Y'].values*factor)
  0%|                                                                                            | 0/4 [00:00<?, ?it/s]<ipython-input-13-d6b00e662556>:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  x = np.int(df[df['TRACK_ID'] == track_id][df['FRAME'] == frame]['POSITION_X'].values*factor)
<ipython-input-13-d6b00e662556>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  y = np.int(df[df['TRACK_ID'

0
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
1
10
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf:

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

 (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
100
im_bf: (4, 140, 140)
101
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
102
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
103
104
105
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
106
107
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf:

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

...

73
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
74
75
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
76
77
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
78
79
8
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)


  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
80
81
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
82
83
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
84
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
85
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
86
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
87
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
88
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
89
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
9
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
90


100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 308.10it/s]

im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
91
92
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
93
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
94
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
95
96
97
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
98
im_bf: (4, 140, 140)
im_bf: (4, 140, 140)
99
im_bf: (4, 140, 140)


In [14]:
import gc

del img, im_bf
gc.collect()

314